<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`: 
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [1]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [2]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [3]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [4]:
# use requests.get() method with the provided static_url and assign the response to a object
response = requests.get(static_url)

Create a `BeautifulSoup` object from the HTML `response`


In [5]:
# Use BeautifulSoup() to create a BeautifulSoup object from a response text content
soup = BeautifulSoup(response.text, 'html.parser')

Print the page title to verify if the `BeautifulSoup` object was created properly 


In [6]:
# Use soup.title attribute
soup.title

<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>

### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [7]:
from io import StringIO

In [ ]:
# Use BeautifulSoup() to create a BeautifulSoup object from a response text content
soup = BeautifulSoup(response.text, 'html.parser')

# Find all tables with class 'wikitable'
tables = soup.find_all('table', {'class': 'wikitable'})

print(f"Found {len(tables)} wikitable(s)")


Found 13 wikitable(s)


In [55]:
# Create dataframe
column_names = [
    'Flight No.', 'Date and time (UTC)', 'Version, Booster', 'Launch site', 'Payload', 
    'Payload mass', 'Orbit', 'Customer', 'Launch outcome', 'Booster landing'
]
df = pd.DataFrame(columns=column_names)

# Convert the HTML table to a pandas DataFrame
for table in tables: 
    # Read html string
    html_string = str(table)
    html_file = StringIO(html_string)
    # Convert table to DataFrame and append to master DataFrame
    sub_table = pd.read_html(html_file)[0]
    if len(sub_table.columns) != 10:
        continue

    sub_table.columns = column_names
    sub_table = sub_table.loc[sub_table['Date and time (UTC)'] != sub_table['Version, Booster']]
    df = pd.concat([df, sub_table], axis=0)


In [56]:
# Function to clean Wikipedia-style references
def clean_cell(cell):
    if isinstance(cell, str):
        return re.sub(r'\[\d+\]', '', cell).strip()
    return cell

df = df.reset_index(drop=True)
df = df.map(clean_cell)

df


,Flight No.,Date and time (UTC),"Version, Booster",Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Booster landing
0,1,"4 June 2010, 18:45",F9 v1.0 B0003.1,"CCAFS, SLC-40",Dragon Spacecraft Qualification Unit,NaN,LEO,SpaceX,Success,Failure (parachute)
1,2,"8 December 2010, 15:43",F9 v1.0 B0004.1,"CCAFS, SLC-40",Dragon demo flight C1 (Dragon C101),NaN,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2,3,"22 May 2012, 07:44",F9 v1.0 B0005.1,"CCAFS, SLC-40",Dragon demo flight C2+ (Dragon C102),"525 kg (1,157 lb)",LEO (ISS),NASA (COTS),Success,No attempt
3,4,"8 October 2012, 00:35",F9 v1.0 B0006.1,"CCAFS, SLC-40",SpaceX CRS-1 (Dragon C103),"4,700 kg (10,400 lb)",LEO (ISS),NASA (CRS),Success,No attempt
4,4,"8 October 2012, 00:35",F9 v1.0 B0006.1,"CCAFS, SLC-40",Orbcomm-OG2,172 kg (379 lb),LEO,Orbcomm,Partial failure,No attempt
...,...,...,...,...,...,...,...,...,...,...
127,117,9 May 2021 06:42,F9 B5 ♺ B1051.10,"CCSFS, SLC-40",Starlink 27 v1.0 (60 satellites),"15,600 kg (34,400 lb)",LEO,SpaceX,Success,Success (drone ship)
128,118,15 May 2021 22:56,F9 B5 ♺ B1058.8,"KSC, LC-39A",Starlink 26 v1.0 (52 Satellites) Capella-6 &T...,"~14,000 kg (31,000 lb)",LEO,SpaceX Capella Space and Tyvak,Success,Success (drone ship)
129,119,26 May 2021 18:59,F9 B5 ♺ B1063.2,"CCSFS, SLC-40",Starlink 28 v1.0 (60 Satellites),"15,600 kg (34,400 lb)",LEO,SpaceX,Success,Success (drone ship)
130,120,3 June 2021 17:29,F9 B5 B1067.1,"KSC, LC-39A",SpaceX CRS-22 (Dragon C209.1),"3,328 kg (7,337 lb)",LEO (ISS),NASA (CRS),Success,Success (drone ship)


Check the extracted column names


We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab. 

Following labs will be using a provided dataset to make each lab independent. 


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a>


<!--
## Change Log
-->


<!--
| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates           |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |
-->


Copyright © 2021 IBM Corporation. All rights reserved.
